In [1]:
import pandas as pd

In [2]:
votos_por_circun = {
    "PRIMERA": {"PAN": 1886007, "PRI": 1162528, "PT": 517448, "PVEM": 704142, "MC": 1774320, "MORENA": 4519464, "PRD": 0},
    "SEGUNDA": {"PAN": 2682634, "PRI": 1521683, "PT": 417610, "PVEM": 997339, "MC": 1330554, "MORENA": 3722803, "PRD": 0},
    "TERCERA": {"PAN": 1320664, "PRI": 945965, "PT": 810826, "PVEM": 1266771, "MC": 960131, "MORENA": 5433356, "PRD": 0},
    "CUARTA":  {"PAN": 2305701, "PRI": 1267427, "PT": 905405, "PVEM": 1128025, "MC": 1234896, "MORENA": 5755609, "PRD": 0},
    "QUINTA":  {"PAN": 1851623, "PRI": 1724639, "PT": 602275, "PVEM": 896009, "MC": 1195620, "MORENA": 4846725, "PRD": 1449176},
}
curules_mr = {"PAN": 31, "PRI": 10, "MC": 1, "MORENA": 182, "PVEM": 40, "PT": 34}

votos_eliminar = {
    "No registrados": 49305,
    "Independientes": 72012,
    "Nulos": 2189171,
}

curules = 200

In [3]:
# ---------------------------------------------
# 1. Suma de votos nacionales por partido
#    (sumando lo que obtuvo en cada circunscripción)
# ---------------------------------------------
votos_por_partido = {
    "PAN": sum(votos_por_circun[c].get("PAN", 0) for c in votos_por_circun),
    "PRI": sum(votos_por_circun[c].get("PRI", 0) for c in votos_por_circun),
    "MC": sum(votos_por_circun[c].get("MC", 0) for c in votos_por_circun),
    "MORENA": sum(votos_por_circun[c].get("MORENA", 0) for c in votos_por_circun),
    "PVEM": sum(votos_por_circun[c].get("PVEM", 0) for c in votos_por_circun),
    "PT": sum(votos_por_circun[c].get("PT", 0) for c in votos_por_circun),
    "PRD": sum(votos_por_circun[c].get("PRD", 0) for c in votos_por_circun),
}

# 2. Total de votos emitidos
#    = votos por partido + votos a eliminar (nulos, no registrados, independientes)
# ---------------------------------------------
votacion_total_emitida = sum(votos_por_partido.values()) + sum(votos_eliminar.values())

# 3. Votación válida emitida
#    = total emitidos - votos nulos - votos no registrados
# ---------------------------------------------
votacion_valida_emitida = votacion_total_emitida - votos_eliminar["Nulos"] - votos_eliminar["No registrados"]

# ---------------------------------------------
# 4. Partidos que superan el umbral nacional (3%)
#    Guardamos en el diccionario partido: proporción
#    ⚠️ Nota: mejor usar >= 0.03 para incluir al que tenga exactamente 3%
# ---------------------------------------------
partidos_con_derecho = {
    p: v / votacion_valida_emitida
    for p, v in votos_por_partido.items()
    if v / votacion_valida_emitida >= 0.03
}

# ---------------------------------------------
# 5. Filtrar los votos de solo esos partidos con derecho
# ---------------------------------------------
votos_partido_con_derecho = {p: v for p, v in votos_por_partido.items() if p in partidos_con_derecho}

# ---------------------------------------------
# 6. Votación nacional emitida para repartir RP
#    = total emitido - todos los eliminados - votos de partidos que no pasaron el umbral
#    (esto es la base para calcular el cociente natural)
# ---------------------------------------------
votacion_nacional_emitida = (
    votacion_total_emitida
    - sum(votos_eliminar.values())
    - sum(v for p, v in votos_por_partido.items() if p not in partidos_con_derecho)
)

# ---------------------------------------------
# 7. Cociente natural
#    = votos nacionales válidos (solo partidos con derecho) / número de curules RP (200)
# ---------------------------------------------
cociente_natural = votacion_nacional_emitida / curules

In [4]:
# ---------------------------------------------
# Asignación por Hare (cociente natural)
#   1) Parte entera: floor(v_i / q)
#   2) Restos mayores: asigna +1 a los restos más grandes hasta completar 'curules'
# ---------------------------------------------

# 1) Parte entera: curules por floor(v_i / q) para cada partido elegible
asignacion_curules = {p: int(v // cociente_natural) for p, v in votos_partido_con_derecho.items()}

# 2) Suma de las partes enteras ya asignadas
curules_asignadas = sum(asignacion_curules.values())

# 3) Curules faltantes para llegar al total (p. ej., 200 nacional o 40 por región)
curules_restantes = curules - curules_asignadas

# 4) Restos: r_i = v_i - floor(v_i / q) * q
#    (equivalente a v_i % q, pero esta forma es más estable en punto flotante)
restos = {
    p: votos_partido_con_derecho[p] - asignacion_curules[p] * cociente_natural
    for p in asignacion_curules
}

# 5) Ordenar por resto descendente (regla de "restos mayores")
#    Sugerencia de robustez: agrega desempate determinista, p.ej.:
#    key=lambda kv: (kv[1], votos_partido_con_derecho[kv[0]], -ORDEN_FIJO[kv[0]])
#    y 'reverse=True'. Aquí mantenemos tu criterio simple.
partidos_ordenados = sorted(restos.items(), key=lambda x: x[1], reverse=True)

# 6) Asignar las curules restantes siguiendo el ranking de restos
#    Propiedad típica de Hare: cada partido suele recibir como mucho +1 por restos (no siempre, pero es lo usual).
for i in range(curules_restantes):
    partido = partidos_ordenados[i][0]
    asignacion_curules[partido] += 1

# ---------------------------------------------
# Totales y tope de sobrerrepresentación
#   - Totales = MR + RP
#   - Capacidad máxima = floor((cuota + 0.08) * 500)
# ---------------------------------------------

# 7) Totales por partido: suma de lo asignado por RP + lo ya obtenido por MR
curules_totales = {
    p: asignacion_curules.get(p, 0) + curules_mr.get(p, 0)
    for p in set(asignacion_curules) | set(curules_mr)
}

# 8) Capacidad máxima por partido con margen +8% (respecto a 500 curules totales)
#    OJO: 'votacion_nacional_emitida' debe ser la base coherente (votos válidos elegibles para tope).
#    Si aplicas tope 300, podrías hacer min(., 300).
curules_maximas = {
    p: int(((v / votacion_nacional_emitida) + 0.08) * 500)
    for p, v in votos_partido_con_derecho.items()
}

# 9) Detectar quienes exceden su capacidad: TOTAL > CAP_MAX
sobrerepresentados = {
    p: curules_totales[p]
    for p in curules_totales
    if curules_totales[p] > curules_maximas.get(p, 0)
}


In [5]:
# Para los partidos sobrerepresentados, dividir su total de votos entre sus diputaciones asignadas (curules_totales)
votos_por_curul_sobrerepresentados = {
    p: votos_partido_con_derecho[p] / (asignacion_curules[p] - (curules_totales[p] - curules_maximas[p]))
    for p in sobrerepresentados
}


votos_por_curul_sobrerepresentados

{'MORENA': 323706.0933333333}

In [6]:
# Calcula la división del total de votos por circunscripción entre el valor de votos_por_curul_sobrerepresentados para cada partido
resultado_division = {
    partido: {
        circ: votos_por_circun[circ][partido] / votos_por_curul_sobrerepresentados[partido]
        for circ in votos_por_circun
    }
    for partido in votos_por_curul_sobrerepresentados
}

# Asigna la parte entera de cada valor en resultado_division a un nuevo diccionario
curules_por_circun_sobrerep_entero = {
    partido: {circ: int(valor) for circ, valor in circuns.items()}
    for partido, circuns in resultado_division.items()
}

# Si la suma de curules es menor a 75, reparte el restante por resto mayor
for partido, circuns in curules_por_circun_sobrerep_entero.items():
    suma_curules = sum(circuns.values())
    if suma_curules < (asignacion_curules[partido] - (curules_totales[partido] - curules_maximas[partido])):
        faltan = (asignacion_curules[partido] - (curules_totales[partido] - curules_maximas[partido])) - suma_curules
        # Ordenar circunscripciones por el valor decimal de resultado_division (resto mayor)
        restos_ordenados = sorted(
            resultado_division[partido].items(),
            key=lambda x: x[1] - int(x[1]),
            reverse=True
        )
        # Asignar +1 a las circunscripciones con mayor resto decimal
        for i in range(faltan):
            circ = restos_ordenados[i][0]
            curules_por_circun_sobrerep_entero[partido][circ] += 1

curules_por_circun_sobrerep_entero

{'MORENA': {'PRIMERA': 14,
  'SEGUNDA': 11,
  'TERCERA': 17,
  'CUARTA': 18,
  'QUINTA': 15}}

In [7]:
nuevas_diputaciones_por_asignar = curules - sum(sum(curules_por_circun_sobrerep_entero[p].values()) for p in sobrerepresentados)

nueva_votacion_nacional_efectiva = votacion_nacional_emitida - sum(votos_partido_con_derecho[p] for p in sobrerepresentados)

cociente_natural_ajustado = nueva_votacion_nacional_efectiva / nuevas_diputaciones_por_asignar



In [8]:
# Obtiene los partidos que no están sobrerepresentados
partidos_no_sobrerepresentados = [p for p in asignacion_curules if p not in sobrerepresentados]
partidos_no_sobrerepresentados

votos_por_curul_no_sobrerep = {
    p: votos_partido_con_derecho[p] / cociente_natural_ajustado
    for p in partidos_no_sobrerepresentados
}

# Calcula la división del total de votos por circunscripción entre el valor de votos_por_curul_no_sobrerep para cada partido
resultado_division_no_sobrerep = {
    partido: {
        circ: votos_por_circun[circ][partido] / votos_por_curul_no_sobrerep[partido]
        for circ in votos_por_circun
    }
    for partido in partidos_no_sobrerepresentados
}

votos_por_curul_no_sobrerep

{'PAN': 39.98150109763561,
 'PRI': 26.35383229457449,
 'MC': 25.849534206072022,
 'PVEM': 19.867269726861704,
 'PT': 12.947862674856182}

In [9]:
votos_por_curul_no_sobrerep

{'PAN': 39.98150109763561,
 'PRI': 26.35383229457449,
 'MC': 25.849534206072022,
 'PVEM': 19.867269726861704,
 'PT': 12.947862674856182}

In [10]:
# Asigna la parte entera de cada valor en resultado_division a un nuevo diccionario
curules_por_circun_entero = {
    partido: {circ: int(valor) for circ, valor in circuns.items()}
    for partido, circuns in resultado_division.items()
}

curules_por_circun_entero

{'MORENA': {'PRIMERA': 13,
  'SEGUNDA': 11,
  'TERCERA': 16,
  'CUARTA': 17,
  'QUINTA': 14}}